# Converting jsons file from the sketchy website

In [20]:
import pandas as pd
import json

In [48]:
import pandas as pd

weather_data = pd.DataFrame(columns=["date"])
for month in range(1,12+1):
    with open(f'data/jsons_weather_monthly/{str(month).zfill(2)}.json') as f:
        data = json.load(f)

    for daily in data["data"]["weather"]:
        hourly_data = []
        for hourly in daily["hourly"]:
            hourly_data.append(pd.DataFrame.from_dict(hourly))
        daily_data = pd.concat(hourly_data)
        daily_data["date"] = daily["date"]
        weather_data = pd.concat([weather_data, daily_data])

weather_data.to_csv("data/weather_hourly.csv")

In [54]:
weather_data

Index(['date', 'time', 'tempC', 'tempF', 'windspeedMiles', 'windspeedKmph',
       'winddirDegree', 'winddir16Point', 'weatherCode', 'weatherIconUrl',
       'weatherDesc', 'precipMM', 'precipInches', 'humidity', 'visibility',
       'visibilityMiles', 'pressure', 'pressureInches', 'cloudcover',
       'HeatIndexC', 'HeatIndexF', 'DewPointC', 'DewPointF', 'WindChillC',
       'WindChillF', 'WindGustMiles', 'WindGustKmph', 'FeelsLikeC',
       'FeelsLikeF', 'uvIndex'],
      dtype='object')

### filter interesting columns

In [66]:
weather_data = weather_data[['date', 'time', 'tempC','windspeedKmph','weatherCode','precipMM','humidity','pressure','cloudcover','WindChillC','WindGustKmph','FeelsLikeC','uvIndex']]

In [67]:
weather_data

,date,time,tempC,windspeedKmph,weatherCode,precipMM,humidity,pressure,cloudcover,WindChillC,WindGustKmph,FeelsLikeC,uvIndex
0,2022-01-01,0,7,6,113,0.0,66,1032,0,6,19,6,1
0,2022-01-01,100,6,6,113,0.0,64,1032,0,5,21,5,1
0,2022-01-01,200,6,6,113,0.0,62,1032,0,4,21,4,1
0,2022-01-01,300,5,6,113,0.0,60,1032,0,4,21,4,1
0,2022-01-01,400,4,6,113,0.0,58,1032,0,3,21,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2022-12-31,1900,8,7,113,0.0,72,1026,3,7,26,7,1
0,2022-12-31,2000,9,7,113,0.0,69,1026,2,8,25,8,1
0,2022-12-31,2100,8,8,113,0.0,66,1027,0,7,26,7,1
0,2022-12-31,2200,8,8,113,0.0,54,1027,2,7,30,7,1


## Combine the datasets in one csv file

In [3]:
import csv
import datetime

# Read weather data
weather_data = {}

with open("data/weather_hourly.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        date = row["date"]
        time = row["time"]
        hour = int(time) // 100
        dt = datetime.datetime.strptime(date, "%Y-%m-%d").replace(hour=hour)
        weather_data[dt] = row

# Read occupation data
occupation_data = {}

with open("data/ocupacio_enginyeria_2022.csv", "r") as f:
    reader = csv.DictReader(f)
    for row in reader:
        if not row["Data inicial"].strip():  # Check if the date string is empty
            continue
        start_date = datetime.datetime.strptime(row["Data inicial"], "%d/%m/%Y")
        start_hour = int(row["Hora inicial"].split(":")[0])
        start_dt = start_date.replace(hour=start_hour)
        occupation_data[start_dt] = row

# Read target data
target_data = {}

with open("data/Consum horari electricitat Enginyeries 2022.csv", "r") as f:
    reader = csv.DictReader(f, delimiter=";")
    for row in reader:
        date = datetime.datetime.strptime(row["Date"], "%d/%m/%Y")
        hour = int(row["Hour"].split(":")[0])
        dt = date.replace(hour=hour)
        target_data[dt] = row

# Merge the data
merged_data = []

for dt in sorted(weather_data.keys()):
    merged_row = weather_data[dt]

    if dt in occupation_data:
        merged_row.update(occupation_data[dt])

    if dt in target_data:
        merged_row.update(target_data[dt])

    merged_data.append(merged_row)

# Save the merged data to a CSV file
with open("merged_data.csv", "w", newline="") as f:
    fieldnames = set()
    for row in merged_data:
        fieldnames.update(row.keys())
    fieldnames = list(fieldnames)
    
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for row in merged_data:
        writer.writerow(row)


ValueError: time data ' ' does not match format '%d/%m/%Y'

# Creating dataset class

In [ ]:
from dataset import energyProject_dataset